In [2]:
''' Importing libraries. '''
import cirq
import numpy as np
import sympy
from sympy.physics.vector.printing import params
import matplotlib.pyplot as plt


Finding low energy configurations of the Ising model using quantum variational circuits. The hamiltonian is :

$ H = - \sum_{<i,j>} Z_i Z_j - \sum_{i} h_i Z_i$

The first sum is over all nearest neighbor pairs and $h$ contains the external field values. Here $i,j$ run over a grid that we define below.

In [7]:
''' Set the dimensions for the grid. '''
n_cols = 3
n_rows = 3

In [15]:
''' Define qubits on the grid.'''
qubits = cirq.GridQubit.rect(n_rows, n_cols)

The variational circuit will have some parameters, which we vary to find the minimum energy (cost function) configuration, using both grid sweep and gradient descent methods.

In [5]:
var_circuit = cirq.Circuit()

The preparatory layer of the circuit has Hadamard gates applied to every qubit, so that the state is a superposition over all possible configurations.

In [8]:
def prep_hadamard(circuit : cirq.Circuit , nrows, ncols):
    for i in range(nrows*ncols):
        circuit.append(cirq.H(qubits[i]))

The circuit is constructed using layers. Each layer has three kinds of operations. Firstly, we have the operation that mimics the first term in the Hamiltonian, the ZZ piece. We also place the grid on a torus.

In [9]:
def zz_apply(circuit : cirq.Circuit, nrows, ncols, gamma):
    for i in range(nrows):
        for j in range(ncols):
            circuit.append(cirq.ZZPowGate(exponent = gamma).on_each((cirq.GridQubit(i,j), cirq.GridQubit((i+1)%nrows,j))))
            circuit.append(cirq.ZZPowGate(exponent = gamma).on_each((cirq.GridQubit(i,j), cirq.GridQubit(i,(j+1)%ncols))))


In [12]:
'''Variational parameters are included using the sympy library.'''
gamma = sympy.Symbol('gamma')
# zz_apply(n_rows, n_cols, gamma)

Then, we have an exponentiation of the second part of the Hamiltonian which has the external field $h$.

In [11]:
'''We take a constant field h with a value 1.3.'''
h = np.full((n_rows,n_cols), 1.3)
def z_apply(circuit : cirq.Circuit, nrows, ncols, h : np.array, gamma):
    for i in range(nrows):
        for j in range(ncols):
            circuit.append(cirq.ZPowGate(exponent = h[i,j]*gamma).on_each(cirq.GridQubit(i,j)))

The third operation in any layer is of the Xpow type, with some variational parameter $\beta$.

In [10]:
beta = sympy.Symbol('beta')

def x_apply(circuit: cirq.Circuit, nrows, ncols, beta):
    for i in range(nrows*ncols):
        circuit.append(cirq.XPowGate(exponent = beta).on_each(qubits[i]))

In [13]:
'''The number of layers determines the number of variational parameters.'''
l = 1

def var_circuit_prep(circuit : cirq.Circuit, beta_params, gamma_params, layers):
    '''This function adds the variational layers to the circuit.'''
    prep_hadamard(circuit, n_rows, n_cols)
    for i in range(layers):
        zz_apply(circuit, n_rows, n_cols, gamma_params[i])
        z_apply(circuit, n_rows, n_cols, h, gamma_params[i])
        x_apply(circuit, n_rows, n_cols, beta_params[i])
    circuit.append(cirq.measure(*qubits))

The variational circuit is shown below :

In [17]:
var_circuit = cirq.Circuit()
var_circuit_prep(var_circuit, beta_params=[beta], gamma_params=[gamma], layers=1)
print(var_circuit)
#simulator = cirq.Simulator()
#results = simulator.run(var_circuit, repetitions=2)
#print(results)
#measurements = results.measurements['q(0, 0),q(0, 1),q(0, 2),q(1, 0),q(1, 1),q(1, 2),q(2, 0),q(2, 1),q(2, 2)']
#print(measurements)

                                                           ┌────────────────┐              ┌─────────────────────┐   ┌───────────────────┐   ┌─────────────────────┐                   ┌─────────────────────┐   ┌───────────────────┐
(0, 0): ───H───ZZ─────────ZZ────────────────────────────────────────────────────ZZ───────────────────────ZZ───────────Z^(1.3*gamma)───────────X^beta──────────────────────────────────────────────────────────────────────────────────────────────────────────────M───
               │          │                                                     │                        │                                                                                                                                                        │
(0, 1): ───H───┼──────────ZZ^gamma───ZZ─────────ZZ──────────────────────────────┼────────────────────────┼────────────────────────────────────ZZ───────────────────────Z^(1.3*gamma)────X^beta────────────────────────────────────────────────────────

In [18]:
# Energy calculation.
# Takes as an input z measurement values.
#m_arr = measurements[1]
#m_arr_new = [1 if m == 0 else -1 for m in m_arr]
#m_arr_new = np.array(m_arr_new).reshape((3,3))
#print(m_arr_new)
#energy = 0
#for i in range(n_rows):
#    for j in range(n_cols):
#        energy = energy + m_arr_new[i,j]*m_arr_new[(i+1)%n_rows,j]
#        energy = energy + m_arr_new[i,j]*m_arr_new[i,(j+1)%n_cols]
#print(energy)

In [19]:
def energy_compute(measurement_outcome, h):
    '''Given the measurement results, this function computes the energy for the configuration.'''
    energy = 0
    m_arr_new = []
    m_arr_new = [1 if m == 0 else -1 for m in measurement_outcome]
    m_arr_new = np.array(m_arr_new).reshape((3,3))
    for i in range(n_rows):
        for j in range(n_cols):
            energy = energy + m_arr_new[i,j]*m_arr_new[(i+1)%n_rows,j]
            energy = energy + m_arr_new[i,j]*m_arr_new[i,(j+1)%n_cols]
            energy = energy + h[i,j]*m_arr_new[i,j]

    return energy



 We perform a manual sweep of the parameter space to find low energy configurations.

In [20]:
sweep_size = 100
sweep = (cirq.Linspace(key='beta', start = 0, stop = 2, length = sweep_size) * cirq.Linspace(key='gamma', start = 0, stop = 2, length = sweep_size))
var_circuit = cirq.Circuit()
simulator = cirq.Simulator()
l = 5
var_circuit_prep(var_circuit, beta_params=[beta for i in range(l)], gamma_params=[gamma for i in range(l)], layers=l)
results = simulator.run_sweep(var_circuit, params = sweep)
energy = []
for result in results:
    measurements = result.measurements['q(0, 0),q(0, 1),q(0, 2),q(1, 0),q(1, 1),q(1, 2),q(2, 0),q(2, 1),q(2, 2)'][0]
    iter_energy = energy_compute(measurements, h)
    energy.append(iter_energy)
    if iter_energy == np.min(energy):
        min_measurements = measurements
    print(f"Parameters : {result.params}")
    print(measurements)
    print(iter_energy)
print(f"The minimum energy is {np.min(energy)} and the configuration giving with this energy is {min_measurements}")

Parameters : cirq.ParamResolver({'beta': 0.0, 'gamma': 0.0})
[0 1 0 1 0 0 0 0 0]
8.5
Parameters : cirq.ParamResolver({'beta': 0.0, 'gamma': 0.020202020202020204})
[1 1 1 0 0 1 0 0 0]
3.3
Parameters : cirq.ParamResolver({'beta': 0.0, 'gamma': 0.04040404040404041})
[1 0 1 0 1 0 1 0 1]
-7.3
Parameters : cirq.ParamResolver({'beta': 0.0, 'gamma': 0.06060606060606061})
[0 1 1 0 0 0 0 1 1]
3.3000000000000007
Parameters : cirq.ParamResolver({'beta': 0.0, 'gamma': 0.08080808080808081})
[0 1 0 1 0 0 0 1 0]
1.9
Parameters : cirq.ParamResolver({'beta': 0.0, 'gamma': 0.10101010101010101})
[1 1 1 1 1 0 1 0 0]
-1.9000000000000001
Parameters : cirq.ParamResolver({'beta': 0.0, 'gamma': 0.12121212121212122})
[1 1 0 0 0 1 0 0 0]
1.9
Parameters : cirq.ParamResolver({'beta': 0.0, 'gamma': 0.1414141414141414})
[0 1 1 0 1 1 0 1 1]
2.1000000000000005
Parameters : cirq.ParamResolver({'beta': 0.0, 'gamma': 0.16161616161616163})
[1 1 1 1 1 1 1 0 0]
-0.49999999999999956
Parameters : cirq.ParamResolver({'beta': 0.

The sweep gives a minimum energy of -9.9. A configuration that has this energy is : $[1,0,1,1,1,0,0,1,1]$.

In [21]:
def grad_energy(beta_init, gamma_init, eps, reps_to_avg, h):
    '''A function that calculates the gradient of the energy. It takes as input :
    beta_init, gamma_init : the parameter values where the derivative is evaluated.
    eps : the difference in the argument when calculating the gradient using discrete differences.
    reps_to_avg : the number of times the circuit is run. The minimum energy from these runs is taken for computing the gradient.
    h : the external field.
    and returns beta and gamma derivatives.
    '''
    sim = cirq.Simulator()
    var_circuit_1 = cirq.Circuit()
    var_circuit_prep(var_circuit_1, beta_params=[beta_init], gamma_params=[gamma_init], layers=1)
    results = sim.run(var_circuit_1, repetitions=reps_to_avg)
    energy = []
    measurements = results.measurements['q(0, 0),q(0, 1),q(0, 2),q(1, 0),q(1, 1),q(1, 2),q(2, 0),q(2, 1),q(2, 2)']
    for i in range(reps_to_avg):
        measurement = measurements[i]
        iter_energy = energy_compute(measurement, h)
        energy.append(iter_energy)
        if iter_energy == np.min(energy):
            min_measurement = measurement
            energy_min = iter_energy

    var_circuit_2 = cirq.Circuit()
    sim = cirq.Simulator()
    var_circuit_prep(var_circuit_2, beta_params=[beta_init + eps], gamma_params=[gamma_init], layers=1)
    results = sim.run(var_circuit_2, repetitions=reps_to_avg)
    energy = []
    measurements = results.measurements['q(0, 0),q(0, 1),q(0, 2),q(1, 0),q(1, 1),q(1, 2),q(2, 0),q(2, 1),q(2, 2)']
    for i in range(reps_to_avg):
        measurement = measurements[i]
        iter_energy = energy_compute(measurement, h)
        energy.append(iter_energy)
        if iter_energy == np.min(energy):
            min_measurement_2 = measurement
            energy_min_2 = iter_energy

    grad_beta_energy = (energy_min_2 - energy_min)/(eps)

    var_circuit_3 = cirq.Circuit()
    sim = cirq.Simulator()
    var_circuit_prep(var_circuit_3, beta_params=[beta_init], gamma_params=[gamma_init + eps], layers=1)
    results = sim.run(var_circuit_3, repetitions=reps_to_avg)
    energy = []
    measurements = results.measurements['q(0, 0),q(0, 1),q(0, 2),q(1, 0),q(1, 1),q(1, 2),q(2, 0),q(2, 1),q(2, 2)']
    for i in range(reps_to_avg):
        measurement = measurements[i]
        iter_energy = energy_compute(measurement, h)
        energy.append(iter_energy)
        if iter_energy == np.min(energy):
            min_measurement_3 = measurement
            energy_min_3 = iter_energy

    grad_gamma_energy = (energy_min_3 - energy_min)/(eps)

    return grad_beta_energy, grad_gamma_energy

In [23]:
#grad_energy(0.65, 0.45, 0.01, 1000, h)

In [26]:
def grad_descent_optimiser(beta_init, gamma_init, eps, reps_to_avg, h):
    '''A function for optimising using a gradient descent. Here beta_init and gamma_init are the initial starting values to start the gradient descent.'''
    b = beta_init
    g = gamma_init
    for i in range(500):
        grad_b, grad_g = grad_energy(b, g, eps, reps_to_avg, h)
        b = b - grad_b*eps
        g = g - grad_g*eps
        if i%100 == 0:
            print(b,g)
    var_circuit = cirq.Circuit()
    sim = cirq.Simulator()
    var_circuit_prep(var_circuit, beta_params=[b], gamma_params=[g], layers=1)
    results = sim.run(var_circuit, repetitions=reps_to_avg)
    energy = []
    measurements = results.measurements['q(0, 0),q(0, 1),q(0, 2),q(1, 0),q(1, 1),q(1, 2),q(2, 0),q(2, 1),q(2, 2)']
    for i in range(reps_to_avg):
        measurement = measurements[i]
        iter_energy = energy_compute(measurement, h)
        energy.append(iter_energy)
        if iter_energy == np.min(energy):
            min_measurement = measurement
            energy_min = iter_energy
    print(f"The minimum energy is : {energy_min} with beta : {b} and gamma : {g}")

In [27]:
grad_descent_optimiser(0.23, 0.87, 0.01, 1000, h)

1.6300000000000012 2.2700000000000005
1.630000000000003 2.270000000000004
1.6299999999999994 2.2700000000000022
1.6300000000000012 2.270000000000006
1.629999999999996 2.2700000000000076
The minimum energy is : -9.900000000000002 with beta : 1.629999999999996 and gamma : 2.270000000000011


We find the same minimum energy again : -9.9, but much faster than an exhaustive sweep.

In [179]:
'''A look at the variational circuit.'''
var_circuit = cirq.Circuit()
simulator = cirq.Simulator()
var_circuit_prep(beta_params=[beta], gamma_params=[gamma], layers=1)
var_circuit

┌────────────────┐              ┌─────────────────────┐   ┌───────────────────┐   ┌─────────────────────┐                   ┌─────────────────────┐   ┌───────────────────┐
(0, 0): ───H───ZZ─────────ZZ────────────────────────────────────────────────────ZZ───────────────────────ZZ───────────Z^(1.3*gamma)───────────X^beta──────────────────────────────────────────────────────────────────────────────────────────────────────────────
               │          │                                                     │                        │
(0, 1): ───H───┼──────────ZZ^gamma───ZZ─────────ZZ──────────────────────────────┼────────────────────────┼────────────────────────────────────ZZ───────────────────────Z^(1.3*gamma)────X^beta────────────────────────────────────────────────────────────────────
               │                     │          │                               │                        │                                    │
(0, 2): ───H───┼─────────────────────┼──────────ZZ^gamma────ZZ──────────────────ZZ^gamma─────────────────┼────────────────────────────────────┼─────────────────────────────────────────ZZ────────────────────────Z^(1.3*gamma)──────────X^beta───────────────────
               │                     │                      │                                            │                                    │                                         │
(1, 0): ───H───ZZ^gamma───ZZ─────────┼──────────ZZ──────────┼────────────────────────────────────────────┼────────────ZZ──────────────────────┼───────Z^(1.3*gamma)────X^beta───────────┼─────────────────────────────────────────────────────────────────────────
                          │          │          │           │                                            │            │                       │                                         │
(1, 1): ───H──────────────┼──────────ZZ^gamma───ZZ^gamma────┼───────ZZ──────────ZZ──────────Z^(1.3*gamma)┼────────────┼────────────X^beta─────┼─────────────────────────────────────────┼─────────────────────────────────────────────────────────────────────────
                          │                                 │       │           │                        │            │                       │                                         │
(1, 2): ───H──────────────┼─────────────────────────────────ZZ^gamma┼───────────ZZ^gamma────ZZ───────────┼────────────ZZ^gamma────────────────┼───────Z^(1.3*gamma)────X^beta───────────┼─────────────────────────────────────────────────────────────────────────
                          │                                         │                       │            │                                    │                                         │
(2, 0): ───H──────────────ZZ^gamma──────────────────────────────────┼───────────────────────┼────────────ZZ^gamma─────ZZ──────────────────────┼─────────────────────────────────────────┼─────────────────────────ZZ─────────────────────Z^(1.3*gamma)───X^beta───
                                                                    │                       │                         │                       │                                         │                         │
(2, 1): ───H────────────────────────────────────────────────────────ZZ^gamma────────────────┼─────────────────────────ZZ^gamma────────────────ZZ^gamma─────────────────ZZ───────────────┼───────Z^(1.3*gamma)─────┼────────────X^beta─────────────────────────────
                                                                                            │                                                                          │                │                         │
(2, 2): ───H────────────────────────────────────────────────────────────────────────────────ZZ^gamma───────────────────────────────────────────────────────────────────ZZ^gamma─────────ZZ^gamma──────────────────ZZ^gamma───────────────Z^(1.3*gamma)───X^beta───
                                                           └────────────────┘ 